# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# imports
import os
from random import choices

from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

In [ ]:
anthropic_url = "https://api.anthropic.com/v1/"
anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)

openai = OpenAI()

In [ ]:
system_prompt = """
You are a Senior Technical Consultant specializing in Data Science, Programming, and Software Engineering. Your objective is to provide high-fidelity, production-ready guidance that bridges the gap between theoretical concepts and practical implementation.

### Operational Guidelines:
1. **Structural Clarity:** Use hierarchical headers, bulleted lists, and horizontal rules to ensure information is scannable and logically organized.
2. **Pedagogical Code:** When providing code, prioritize PEP 8 (Python) or relevant style guides. Include concise inline comments for complex logic and a brief "How it works" summary below the block.
3. **The "Why" Behind the "How":** For debugging or refactoring, always diagnose the root cause (the "why") before presenting the solution (the "how").
4. **Guardrails & Best Practices:** Proactively identify "footguns" (common pitfalls), security vulnerabilities, or performance bottlenecks associated with the solution.
5. **Epistemic Humility:** If a query is ambiguous, present the most likely interpretations as distinct scenarios. If a technical detail is outside your high-confidence range, explicitly state the uncertainty.
6. **Documentation First:** Reference official documentation (e.g., MDN, PyTorch Docs, PEPs) to ground your answers in authoritative sources.

### Tone and Style:
- Maintain a professional, objective, and intellectually honest tone.
- Avoid fluff; prioritize technical density and "signal-to-noise" ratio.
- Prioritize accuracy and architectural integrity over brevity.
"""

In [ ]:
# Here we define the models that the user can select from the dropdown
PROVIDER_OPTIONS = {
    "OpenAI": [
        "gpt-4.1-mini",
        "gpt-5.1"
    ],
    "Anthropic": [
        "claude-haiku-4-5-20251001",
        "claude-sonnet-4-6"
    ]
}

PROVIDER_MAPPING = {
    "OpenAI": openai,
    "Anthropic": anthropic
}

In [ ]:
# Streaming chat function
def chat(provider, model, history):
    messages = (
            [{"role": "system", "content": system_prompt}]
            + [{"role": h["role"], "content": h["content"]} for h in history]
    )

    stream = PROVIDER_MAPPING[provider].chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )

    response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            response += chunk.choices[0].delta.content
            yield history + [{"role": "assistant", "content": response}]

In [ ]:
#Gradio Interface
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
    with gr.Row():
        provider_dropdown = gr.Dropdown(
            choices=["OpenAI", "Anthropic"],
            value="OpenAI",
            label="Select Provider"
        )

        model_dropdown = gr.Dropdown(
            label="Select Model",
            choices=PROVIDER_OPTIONS["OpenAI"],
            value=PROVIDER_OPTIONS["OpenAI"][0],
        )


        # A function to update the models dropdown when the provider changes
        def update_models(provider):
            choices = PROVIDER_OPTIONS.get(provider, [])
            value = choices[0] if len(choices) > 0 else None
            return gr.Dropdown(choices=choices, value=value, label="Select Model")


        provider_dropdown.change(
            fn=update_models,
            inputs=[provider_dropdown],
            outputs=[model_dropdown]
        )

    with gr.Row():
        message_box = gr.Textbox(
            label="Chat with AI Assistant: (Shift + Enter) to send the message",
            lines=10
        )

    def handle_submit(user_message, history):
        # Add user message to history
        history += [{"role": "user", "content": user_message}]
        return "", history


    message_box.submit(
        handle_submit,
        inputs=[message_box, chatbot],
        outputs=[message_box, chatbot]
    ).then(
        chat,
        inputs=[provider_dropdown, model_dropdown, chatbot],
        outputs=[chatbot]
    )

    # Examples
    gr.Examples(
        examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ],
        inputs=message_box
    )

ui.launch(inbrowser=True)

My goals for this assignment:
 - To enable a user to use different providers and different models within a selected provider.
 - To be able to stream responses from the different providers